In [1]:
print("Hello World")

Hello World


In [3]:
import nltk
import pandas as pd
import numpy as np

In [5]:
dataset=pd.read_csv("E:\DRIVE D\SOHAM ENG COLL\ENGINEERING NOTES\SEM 5\ML\SMSSpamCollection (9) (2).tsv.txt", sep="\t", header=None)
dataset.columns=['label', 'body_text']
dataset.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xba in position 110: invalid start byte

In [6]:
import numpy as np
from scipy.optimize import minimize

# Simulated data: Each row represents a person's responses (0 or 1) to items
responses = np.array([[1, 0, 1, 1],
                      [0, 1, 1, 0],
                      [1, 1, 0, 0]])

# Initial parameter values (difficulty parameters for each item)
initial_params = np.zeros(responses.shape[1])

# Rasch model function (1PL model)
def rasch_model(params, *args):
    ability = args[0]
    responses = args[1]
    theta = ability
    b = params  # difficulty parameters
    logits = theta - b
    probabilities = 1 / (1 + np.exp(-logits))
    log_likelihood = np.sum(responses * np.log(probabilities) + (1 - responses) * np.log(1 - probabilities))
    return -log_likelihood

# Estimate parameters using Maximum Likelihood Estimation
estimated_params = minimize(rasch_model, initial_params, args=(0, responses), method='L-BFGS-B')

# Extract estimated difficulty parameters
difficulty_params = estimated_params.x

print("Estimated difficulty parameters:", difficulty_params)


Estimated difficulty parameters: [-0.6931466  -0.69314647 -0.69314663  0.69314626]


In [7]:
class BKTModel:
    def __init__(self, num_skills):
        self.num_skills = num_skills
        self.P_L = np.full(num_skills, 0.1)  # Initial probability of knowing the skill
        self.P_T = np.full(num_skills, 0.3)  # Probability of transitioning from not knowing to knowing
        self.P_G = np.full(num_skills, 0.2)  # Probability of guessing correctly when not knowing
        self.P_S = np.full(num_skills, 0.1)  # Probability of slipping up when knowing

    def update(self, responses):
        for i, response in enumerate(responses):
            P_L_new = (self.P_L[i] * (1 - self.P_T[i])) / ((self.P_L[i] * (1 - self.P_T[i])) + ((1 - self.P_L[i]) * self.P_G[i]))
            P_L_new_knowing = (self.P_L[i] * self.P_T[i]) / ((self.P_L[i] * self.P_T[i]) + ((1 - self.P_L[i]) * (1 - self.P_S[i])))
            if response == 1:  # The learner answered correctly
                self.P_L[i] = P_L_new_knowing
            else:
                self.P_L[i] = P_L_new

# Example usage:
responses = [[1, 0, 1, 0],
             [0, 1, 1, 1],
             [1, 0, 0, 1]]

bkt_model = BKTModel(num_skills=len(responses[0]))
for response_set in responses:
    bkt_model.update(response_set)

print("Updated Knowledge Probabilities (P_L) for each skill:", bkt_model.P_L)


Updated Knowledge Probabilities (P_L) for each skill: [0.04142012 0.31210191 0.04142012 0.04142012]


In [8]:
import numpy as np
from scipy.optimize import minimize

class IRTModel:
    def __init__(self, num_items):
        self.num_items = num_items

    def likelihood(self, params, responses):
        theta, discrimination, difficulty = params
        logits = discrimination * (theta - difficulty)
        probabilities = 1 / (1 + np.exp(-logits))
        likelihoods = responses * np.log(probabilities) + (1 - responses) * np.log(1 - probabilities)
        return -np.sum(likelihoods)

    def fit(self, responses):
        initial_params = [0, 1, 0]  # Initial values for theta, discrimination, and difficulty
        bounds = [(-4, 4), (0.01, 5), (-3, 3)]  # Bounds for optimization parameters
        result = minimize(self.likelihood, initial_params, args=(responses,), bounds=bounds)
        return result.x

# Example usage:
responses = np.array([[1, 0, 1, 0],
                      [0, 1, 1, 1],
                      [1, 0, 0, 1]])

irt_model = IRTModel(num_items=responses.shape[1])
params = irt_model.fit(responses)
estimated_theta = params[0]

print("Estimated Theta (Student's Ability):", estimated_theta)
print("Estimated Parameters (theta, discrimination, difficulty):", params)


Estimated Theta (Student's Ability): 0.16791001732619873
Estimated Parameters (theta, discrimination, difficulty): [ 0.16791002  1.00193552 -0.16791005]
